# Michał Szczurek - lab 5

In [1]:
import math
from collections import Counter
import numpy as np
from sklearn.cluster import DBSCAN
from functools import lru_cache
from enum import Enum
import copy

# 1. Implementacja metryk

### 1.1 Funkcje pomocnicze związane z n-gramami

In [2]:
def make_ngram_vec(x, N=2):
    vec = {}
    for i in range(len(x)-N+1):
        if x[i:i+N] in vec:
            vec[x[i:i+N]] += 1
        else:
            vec[x[i:i+N]] = 1
    return vec

In [3]:
def vec_len(vec):
    res = 0
    for key, value in vec.items():
        res += value**2
    return math.sqrt(res)

In [4]:
def make_ngram_set(x, N=2):
    res = set()
    for i in range(len(x)-N+1):
        res.add(x[i:i+N])
    return res

### 1.2 metryka LCS

In [5]:
@lru_cache(maxsize=None)
def LCS_m(x, y):
    tab = [[None for j in range(len(y)+1)] for i in range(len(x)+1)]
    max_lcs = 0
    for i in range(len(x)+1):
        for j in range(len(y)+1):
            if i == 0 or j == 0:
                tab[i][j] = 0
            else:
                if x[i-1] == y[j-1]:
                    tab[i][j] = 1 + tab[i-1][j-1]
                else:
                    tab[i][j] = 0
            if tab[i][j] > max_lcs:
                max_lcs = tab[i][j]
    return 1 - max_lcs/max(len(x), len(y))

In [6]:
LCS_m("katzaaba", "bbktzazb")

0.625

### 1.3 metryka Euklidesowa

In [7]:
@lru_cache(maxsize=None)
def eucledian_m(x, y, N=2):
    x_vec = make_ngram_vec(x, N)
    y_vec = make_ngram_vec(y, N)
        
    res = 0 
    
    for k in x_vec:
        if k in y_vec:
            res += (x_vec[k]- y_vec[k])**2
        else:
            res += (x_vec[k])**2 
            
    for k in y_vec: 
        if k not in x_vec:
            res += y_vec[k]**2
            
    return math.sqrt(res)

In [8]:
eucledian_m("katzaaba", "bbktzazb")

3.1622776601683795

### 1.4 metryka Levenshteina

In [32]:
@lru_cache(maxsize=None)
def levenshtein_m(x, y):
    
    def delta(a, b):
        if a == b:
            return 0
        else:
            return 1
    
    edit_table = np.empty((len(x)+1, len(y)+1))
    for i in range(len(x)+1):
        edit_table[i, 0] = i
    for j in range(len(y)+1):
        edit_table[0, j] = j
        
    for i in range(len(x)):
        k = i + 1
        for j in range(len(y)):
            l = j + 1
            edit_table[k,l] = min(edit_table[k-1,l]+1, 
                                  edit_table[k,l-1]+1, 
                                  edit_table[k-1, l-1] + delta(x[i], y[j]))
    if len(x) == len(y) == 0:
        return 0
    return edit_table[len(x), len(y)] / max(len(x), len(y))

In [33]:
levenshtein_m("katzaaba", "bbktzazb")

0.625

### 1.5 Metryka cosinusowa

In [11]:
@lru_cache(maxsize=None)
def cosine_m(x, y, N=2):
    x_vec = make_ngram_vec(x, N)
    y_vec = make_ngram_vec(y, N)

    dot_product = 0
    for key, value in x_vec.items():
        if key in y_vec:
            dot_product += value * y_vec[key]
    if (vec_len(x_vec) * vec_len(y_vec)) == 0: 
        return 1
    return 1 -  dot_product/(vec_len(x_vec) * (vec_len(y_vec)))

In [12]:
cosine_m("katzaaba", "bbktzazb")

0.7142857142857143

### 1.6 Współczynnik DICE

In [13]:
@lru_cache(maxsize=None)
def DICE_coef(x, y, N=2):
    x_set = make_ngram_set(x, N)
    y_set = make_ngram_set(y, N)
    if (len(x_set) + len(y_set)) == 0:
        return 0
    numerator = len(x_set.intersection(y_set))
    return 1 - (2 * numerator / (len(x_set) + len(y_set)))     

In [14]:
DICE_coef("katzaaba", "bbktzazb")

0.7142857142857143

# 2 Ocena jakości klasteryzacji

### 2.1 Funkcje pomocnicze

In [15]:
def find_centroid(cluster, metric, *args):
    dists = [0 for i in cluster]
    for i in range(len(cluster)):
        for j in range(i):
            dist = metric(cluster[i], cluster[j], *args)
            dists[i] += dist
            dists[j] += dist      
    min_dist = 10**100 # inf
    res = -1
    for i in range(len(dists)):
        if dists[i] < min_dist:
            min_dist = dists[i]
            res = i
    return cluster[res]      

In [16]:
def avg_dist(cluster, metric, *args):
    if len(cluster) < 2:
        return 0
    if len(cluster) == 2:
        return metric(cluster[0], cluster[1], *args)
    
    dist_sum = 0
    for i in range(len(cluster)):
        for j in range(i):
            dist_sum += metric(cluster[i], cluster[j], *args)
    return dist_sum / ((len(cluster)-2) * (len(cluster)-1))

In [17]:
def cluster_diameter(cluster, metric, *args):
    max_dist = -1
    for i in range(len(cluster)):
        for j in range(i):
            max_dist = max(metric(cluster[i], cluster[j], *args), max_dist)
    return max_dist  

### 2.1 Indeks Daviesa-Bouldina

In [18]:
def DB_index(clusters, metric, *args):
    centroids = [find_centroid(c, metric, *args) for c in clusters]
    avg_dists = [avg_dist(c, metric, *args) for c in clusters]
    n = len(clusters)
    tab = [max([(avg_dists[i] + avg_dists[j]) / metric(centroids[i], centroids[j], *args) \
                for i in range(n) if i != j]) for j in range(n)]
    return sum(tab) / n

### 2.2 Indeks Dunna

In [19]:
def dunn_index(clusters, metric, *args):
    centroids = [find_centroid(c, metric, *args) for c in clusters]
    diameters = [cluster_diameter(c, metric, *args) for c in clusters]
    
    min_dist = 10**100 
    for i in range(len(centroids)):
        for j in range(i):
            min_dist = min(metric(centroids[i], centroids[j], *args), min_dist)
    max_size = max(diameters)
    return min_dist / max_size

# 3. Stoplista

In [20]:
class StopList:
    
    def __init__(self, text, frequency):
        words = []
        for line in text:
            words += (line.split())
        counter = Counter(words)
        words_sum = len(words)
        self.common = {key for key, value in counter.items() if value >= frequency * words_sum}
        
    def remove_common(self, text):
        res = []
        for line in text:
            res.append(' '.join([w for w in line.split() if w not in self.common]))
        return res

In [21]:
stop_list = StopList(["ala ma kota", "ala miała kot", "kot franek"], 0.2)

In [22]:
stop_list.remove_common(["ala ma kota", "ala miała kot", "kot franek"])

['ma kota', 'miała', 'franek']

# 4. Klasteryzacja

### 4.1 Funkcje pomocnicze

In [23]:
def read_text(file, n):
    with open(file, 'r', encoding='utf-8') as f:
        text = f.read().splitlines()
    return text[:n]

In [24]:
def save_clusters(file, clusters):
    with open(file, 'w', encoding='utf-8') as f:
        for c in clusters:
            for line in c:
                f.write(line+'\n')
            f.write('============\n\n')

Poniższa funkcja wypisuje n klastrów zawierających więcej niż 1 element.

In [25]:
def print_clusters(clusters, n=10):
    i = j =0
    while j < n and i < len(clusters):
        c = clusters[i]
        if len(c) != 1:    
            for line in c:
                print(line)
            print('============\n')
            j += 1
        i += 1

In [26]:
def perform_clustering(text, metric_f, eps, stop_list_frequency=None, *args):
    working_text = text
    if stop_list_frequency is not None:
        stop_list = StopList(text, stop_list_frequency)
        working_text = stop_list.remove_common(text)
        
    def metric(x, y):
        i, j = int(x[0]), int(y[0])
        return metric_f(working_text[i], working_text[j], *args)
    
    X = np.arange(len(working_text)).reshape(-1, 1)
    clustering = DBSCAN(metric=metric, min_samples=1, eps=eps).fit_predict(X)
    #print(clustering)
    clusters = [[] for i in range(max(clustering) + 1)]
    for i in range(len(clustering)):
        clusters[clustering[i]].append(text[i])
    return clusters

# 5. Test klasteryzacji

### 5.1 Funkcje pomocn icze

In [130]:
def perform_tests(to_test, stop_list, res_v, eps_v):
    if stop_list:
        frequency = 0.01
    else:
        frequency = None
    for i in range(len(to_test)):
        clusters = perform_clustering(text, to_test[i], eps_v[i], frequency)
        print("Funkcja: ",to_test[i].__name__)
        print("Epsilon: ",eps_v[i])
        print("Indeks Daviesa-Bouldina: ", DB_index(clusters, to_test[i]))
        print("Indeks Dunna: ", dunn_index(clusters, to_test[i]))
        print("========\n")
        if stop_list:
            save_clusters(to_test[i].__name__ + "_stop.txt", clusters)
        else:
            save_clusters(to_test[i].__name__ + "_no_stop.txt", clusters)
        res_v.append(clusters)

In [131]:
def print_all(res_v, to_test):
    for i in range(len(res_v)//2):
        print("Klasteryzacja przy pomocy " + to_test[i].__name__ + " bez stoplisty:\n\n")
        print_clusters(res_v[i])
        print("**********************************************************************\n\n\n")
    for i in range(len(res_v)//2 ,len(res_v)):
        print("Klasteryzacja przy pomocy " + to_test[i - len(res_v)//2].__name__ + " ze stoplistą:\n\n")
        print_clusters(res_v[i])
        print("**********************************************************************\n\n\n")

### 5.2 Testy

Testy wykonano dla pierwszych 300 linii teskstu.

In [132]:
text = read_text("lines.txt", 300)

In [141]:
to_test = [cosine_m, DICE_coef, LCS_m, levenshtein_m]
res_v = []
eps_v = [0.3, 0.3, 0.7, 0.45]
res_vec_stop = []
print("Test bez stoplisty\n\n")
perform_tests(to_test, False, res_v, eps_v)  
print("\nTest ze stoplistą usuwającą wyrazy częstsze niż 0.01 * liczba wyrazów\n\n")
perform_tests(to_test, True, res_v, eps_v)

Test bez stoplisty


Funkcja:  cosine_m
Epsilon:  0.3
Indeks Daviesa-Bouldina:  0.47336466196968524
Indeks Dunna:  0.5557070610373043

Funkcja:  DICE_coef
Epsilon:  0.3
Indeks Daviesa-Bouldina:  0.47043960799801005
Indeks Dunna:  0.6840219727728685

Funkcja:  LCS_m
Epsilon:  0.7
Indeks Daviesa-Bouldina:  1.2476753251118504
Indeks Dunna:  0.8203918722786648

Funkcja:  levenshtein_m
Epsilon:  0.45
Indeks Daviesa-Bouldina:  0.9338007462411682
Indeks Dunna:  0.6823529411764705


Test ze stoplistą usuwającą wyrazy częstsze niż 0.01 * liczba wyrazów


Funkcja:  cosine_m
Epsilon:  0.3
Indeks Daviesa-Bouldina:  0.4649200757617744
Indeks Dunna:  0.5331330270508716

Funkcja:  DICE_coef
Epsilon:  0.3
Indeks Daviesa-Bouldina:  0.4769826361889164
Indeks Dunna:  0.7552742616033755

Funkcja:  LCS_m
Epsilon:  0.7
Indeks Daviesa-Bouldina:  1.3588212804243278
Indeks Dunna:  0.7894034686487518

Funkcja:  levenshtein_m
Epsilon:  0.45
Indeks Daviesa-Bouldina:  0.9338007462411682
Indeks Dunna:  0.6823529411

Wartości epsilon dobrałem tak, by stworzone klastry w jak największym stopniu odopowiadały tym w pliku dołączonym do zadania oraz by zgadzały się z moją oceną (uzasadnionom np nazwami firm). Wymagało to, większego epsilonu dla metryki LCS i Levenshtein'a, co spowodowało Indeks Daviesa-Bouldina  i Dunna był największy dla LCS. Pierwsze zjawisko jest negatywne, drugie zaś pozytywne. Użycie stop listy nie spowodowało drastycznych zmian w podziale na klastry. Zmieniło jednak wartości indeksów:
* Zmniejszyło oba indeksy dla meteryki cosinusowej (zmiana na pozytywno-negatywna)  
* Zwiększyło indeks Dunna dla współczynnika DICE (zmiana na +)   
* Zmniejszyło indeks Dunna i zwiększyło ineks Daviesa-Bouldina dla metyki LCS (zmiany na -)  
* praktycznie nie zmieniło indeksów dla metryki Levenshteina

Ranking klasteryzacji wg idneksu Daviesa-Bouldina:

1. Metryka cosinusowa (ze stoplistą)  
2. Współczynnik DICE (bez stoplisty)  
3. Metryka cosinusowa (bez stoplisty)  
4. Współczynnik DICE (ze stoplistą)
5. Metryka Levenshteina (z i bez stoplisty)  
7. Metryka LCS (bez stoplisty)    
8. Metryka LCS (ze stoplistą)   

Ranking klasteryzacji wg idneksu Dunna:

1. Metryka LCS (bez stoplisty) 
2. Metryka LCS (ze stoplistą)  
3. Współczynnik DICE (ze stoplistą)  
4. Współczynnik DICE (bez stoplisty) 
5. Metryka Levenshteina (z i bez stoplisty)  
7. Metryka cosinusowa (ze stoplistą)
8. Metryka cosinusowa (bez stoplisty)  

Czas wykonywania się klasteryzacji przy użyuciu poszczególnych metryk zachowywał (według moich obserwacji) nasępującą relację:  
współczynnik DICE  < metryka cosinusowa << metryka LCS << mtryka Levenshtein'a

Zapamiętywanie wyników funkcji w znacznym stopniu przyspiesza kolejne wywołania funkcji. (w szczególności pomaga to testować różne epsilon, jako że wartości metryk nie zależą od epsilon)

Myślę, że następujące czynności mogłyby polepszyć klasteryzację:
* zamienienie wszystkich liter na litery jednakowej wielkości / uwzględnienie mniejszych odległości dla tej samej litery o róznej wielkości w metrykach - Często nazwy firm były pisane raz dużymi literami a raz małymi. Z punktu widzenia podziału na grupy, nie powinno mieć to znaczenia.
* Zwiększenie wagi odległości między pierwszym/ pierwszymi wyrazami linii. Zazwyczaj pierwszy wyraz to nazwa firmy, a więc czynnik, który jest ważniejszy niż np email (ten może się zmieniać).

#### W dalszej części sprawozdania znajdują się wydruki dla poszczególnych klasteryzacji

Wydruki obejmują przykładowe 10 klastrów o rozmiarze (w sensie ilości elementów) większym od 1.  
Pełne wyniki załączone są w pliku wyniki.zip

In [125]:
print_all(res_v, to_test)

Klasteryzacja przy pomocy cosine_m bez stoplisty:


''SSONTEX''  Sp.ZO.O.IMPORT-EXPORTUL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669
''SSONTEX''SP.ZO.O.IMPORT-EXPORT UL:PRZECLAWSKA 5 03-879 WARSZAWA,POLAND NIP 113-01-17-669 TEL./FAX.:0048(022)217 6532--
"SSONTEX" SP.ZO.O IMPORT-EXPORT 03-879 WARSZAWA UL PRZECLAWSKA 5 NIP:113-01-17-669

''TOPEX SP. Z O.O.'' SPOLKA KOMANDYTOWA UL. POGRANICZNA 2/4  02-285 WARSZAWA POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4, 02-285 WARSZAWA.POLAND
"TOPEX SP.Z.O.O."SP.K. UL.POGRANICZNA 2/4,02-285 WARSZAWA POLAND
"TOPEX SP.Z O.O."SP.K. UL,POGRANICZNA 2/4 02-285 WARSZAWA
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2/4,02--285 WARSZAWA
"TOPEX SP. Z O. O." SP. K. UL.POGRANICZNA 2/4,02-285 WARSZAWA,POLAND
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2/4,02-285 WARSZAWA T:0048 225730397 F:0048 2257 30400
"TOPEX SP.Z O.O."SP.K. UL.POGRANICZNA 2/4 02-285 WARSZAWA POLAND
"TOPEX SP. Z O.O." SP.K. UL.POGRANICZNA 2/4 02-285 WARSZAWA POLAND
"TOPEX SP.Z O.O." SPOLKA 